Final project     
Data science for electrical energy systems  

Analyzing Electricity Market data using Machine Learning 

The electricity market is influenced by various factors such as demand, renewable energy generation, 
weather conditions, and fuel prices. Accurate forecasting and analysis are crucial for market participants to 
optimize operations and make informed decisions. 

Objective 
Work with Spanish historical electricity market data (find it on the ESIOS-REE webpage) to: 
• Supervised Learning (Regression): Predict the day-ahead spot electricity price in Spain. 
• Unsupervised Learning (Clustering): Group daily electricity price profiles into clusters to identify 
patterns. 
• Principal Component Analysis (PCA): Reduce data dimensionality and test its effect on your 
models. 
Tasks and guidelines 

TASK 1) EDA: Exploratory Data Analysis (EDA) of potential input variables for your forecasting model.  

TASK 2) Develop a machine learning model (supervised) to forecast the day ahead electricity price in Spain.  
To obtain the target data, access the Esios website and download historical spot electricity prices. Possible 
input data can also be searched on this website. Some general guidelines for developing your model are 
detailed below. 
1) Understand your data set. Use descriptive statistics and data visualization tools. 
2) Data analysis of the data. Important! Prepare your dataset. Clean it and transform it, if needed.  
3) Think about possible feature selection for your model.  
4) Split the dataset into training and test. 
5) Train your model using different ML. 
6) Evaluate your model, adjust hyperparameters and retrain it if needed +. 
7) Validate your model using the following regression metrics: RMSE, MAE and R2. 

TASK 3) Create a clustering model to group the daily SPOT electricity price profiles in different clusters.  
Use two clustering techniques (e.g., K-means, hierarchical clustering, DBSCAN…) to identify patterns or 
segments in the data based on historical electricity prices and other features, if necessary. Compare the 
results. Are they similar? 

TASK 4) Test the regression and clustering methods using PCA 
Submission Requirements: 
Jupyter Notebook: 
• Include all your code for data analysis, model development, and evaluation. Inside this file, 
SEPARATE the different Tasks. 
• Ensure the notebook is well-organized with clear explanations and comments. 
Report (PDF): 
• Summarize your methodology, findings, and conclusions. It should have an index and sections 
explaining the main key findings. Include key visualizations and tables. 
• Discuss any challenges faced and how you addressed them.  

Deadline: Final Report to be submitted before the end of the day on December 20th. Be sure to cite external 
resources or libraries you use. Your work should be original and reflect your understanding of machine 
learning concepts applied to real-world data.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [28]:
#GM

# Carga dataset

dataset19 = pd.read_csv('export_PrecioMercadoSPOTDiario_2019.csv', sep=';')
dataset20 = pd.read_csv('export_PrecioMercadoSPOTDiario_2020.csv', sep=';')
dataset21 = pd.read_csv('export_PrecioMercadoSPOTDiario_2021.csv', sep=';')
dataset22 = pd.read_csv('export_PrecioMercadoSPOTDiario_2022.csv', sep=';')
dataset23 = pd.read_csv('export_PrecioMercadoSPOTDiario_2023.csv', sep=';')



In [61]:
#TASK 1

data19 = dataset19
data20 = dataset20
data21 = dataset21
data22 = dataset22
data23 = dataset23

datasets = [data19, data20, data21, data22, data23]

print(data19.dtypes)

#Casting datetime
for data in datasets:
        data['datetime'] = pd.to_datetime(data['datetime'], utc=True)

print('\n')
print(data19.dtypes)

# EDA
def eda(dataframe):

    data19 = dataframe
    print(data19.shape)

    print('\n')
    print(data19.isna().sum())
    print(data19.describe())

    print('\n')
    print(data19['datetime'].nunique())
    print(data19['geoid'].nunique())
    print(data19['geoname'].nunique())
    print(data19['id'].nunique())
    print(data19['name'].nunique())
    print(data19['value'].nunique())
    data19.head()

for data in datasets:
    eda(data)

id                        int64
name                     object
geoid                     int64
geoname                  object
value                   float64
datetime    datetime64[ns, UTC]
dtype: object


id                        int64
name                     object
geoid                     int64
geoname                  object
value                   float64
datetime    datetime64[ns, UTC]
dtype: object
(8760, 6)


id          0
name        0
geoid       0
geoname     0
value       0
datetime    0
dtype: int64
           id   geoid        value
count  8760.0  8760.0  8760.000000
mean    600.0     3.0    47.681316
std       0.0     0.0    10.881934
min     600.0     3.0     0.030000
25%     600.0     3.0    42.010000
50%     600.0     3.0    48.950000
75%     600.0     3.0    54.530000
max     600.0     3.0    74.740000


8760
1
1
1
1
2920
(8760, 6)


id          0
name        0
geoid       0
geoname     0
value       0
datetime    0
dtype: int64
           id   geoid        valu